In [1]:
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import xml.etree.ElementTree as et
import numpy as np

In [2]:
df = pd.read_csv('train/golden_truth.txt', sep=" ")
df

,id,label
0,subject671,1
1,subject9917,1
2,subject6238,1
3,subject8581,1
4,subject7238,1
...,...,...
335,subject2182,0
336,subject9829,0
337,subject3270,0
338,subject6464,0


In [3]:
def parse_XML(xml_file, df_cols): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

train = pd.DataFrame()

for index, row in df.iterrows():
    df = parse_XML('train/data/'+row['id']+'.xml', ['ID','DATE', 'TEXT'])
    df['truth'] = row['label']
    df['ID'] = row['id']
    train = pd.concat([train, df])

In [4]:
train.dropna(inplace=True, thresh=3)
train = train.replace(to_replace='None', value=np.nan).dropna()

In [5]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text


train['TEXT'] = train['TEXT'].apply(lambda x: remove_punctuation(x), 1)
train['TEXT'] = train['TEXT'].apply(lambda x: tokenizer.tokenize(x.lower()))
train['TEXT'] = train['TEXT'].apply(lambda x: remove_stopwords(x))
train['TEXT'] = train['TEXT'].apply(lambda x: word_lemmatizer(x))
train['TEXT'] = train['TEXT'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
train['TEXT'] = train['TEXT'].apply(lambda elem: re.sub(r"\d+", "", elem))

In [6]:
train = train.sample(frac=1).reset_index(drop=True)
train.dropna(inplace=True)
del train['DATE']
del train['ID']
train.to_csv('train.csv', header = True, index=False)